In [4]:
import cv2
import os

directory_path = 'MCI/video_sequence'

In [6]:
import cv2
import os
import numpy as np

directory_path = 'MCI/video_sequence'
frame_list = []

width = height = 32

for filename in os.listdir(directory_path):
    if filename.endswith(".avi"):
        video_path = os.path.join(directory_path, filename)
        cap = cv2.VideoCapture(video_path)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to grayscale
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Resize the frame
            resized_frame = cv2.resize(gray_frame, (width, height))

            # Add the frame to the list
            frame_list.append(resized_frame)

        cap.release()

cv2.destroyAllWindows()

# Checking the shape of the first frame in the list
print(len(frame_list))
print(frame_list[0].shape)  # Should output: (32, 32)

25200
(32, 32)


In [7]:
# Convert list to NumPy array and add an extra dimension
X_train = np.array(frame_list, dtype=np.float32)[..., np.newaxis]

# Normalize the data to the range [0, 1]
X_train = X_train / 255.0

print(X_train.shape)  # Should output: (num_frames, 32, 32, 1)


(25200, 32, 32, 1)


In [8]:
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, LeakyReLU, BatchNormalization, Reshape, Flatten, Dense
from tensorflow.keras.models import Model

def build_autoencoder(input_shape):
    # Encoder
    input_layer = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), strides=2, padding='same')(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(64, (3, 3), strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    # Flatten and bottleneck
    volume_size = x.shape
    x = Flatten()(x)
    latent = Dense(100)(x)  # Latent vector

    # Decoder
    x = Dense(np.prod(volume_size[1:]))(latent)
    x = Reshape((volume_size[1], volume_size[2], volume_size[3]))(x)

    x = Conv2DTranspose(64, (3, 3), strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv2DTranspose(32, (3, 3), strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    output_layer = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_layer, output_layer)
    return autoencoder

In [9]:
autoencoder = build_autoencoder(X_train.shape[1:])
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32)


Epoch 1/50
788/788 [==============================] - 14s 17ms/step - loss: 0.0064
Epoch 2/50
788/788 [==============================] - 16s 20ms/step - loss: 0.0040
Epoch 3/50
788/788 [==============================] - 15s 20ms/step - loss: 0.0037
Epoch 4/50
788/788 [==============================] - 15s 19ms/step - loss: 0.0034
Epoch 5/50
788/788 [==============================] - 15s 19ms/step - loss: 0.0033
Epoch 6/50
788/788 [==============================] - 15s 20ms/step - loss: 0.0032
Epoch 7/50
788/788 [==============================] - 15s 19ms/step - loss: 0.0031
Epoch 8/50
788/788 [==============================] - 15s 19ms/step - loss: 0.0030
Epoch 9/50
788/788 [==============================] - 15s 19ms/step - loss: 0.0029
Epoch 10/50
788/788 [==============================] - 15s 19ms/step - loss: 0.0029
Epoch 11/50
788/788 [==============================] - 15s 19ms/step - loss: 0.0029
Epoch 12/50
788/788 [==============================] - 15s 19ms/step - loss: 0.0028
E

In [12]:
decoded_frames = autoencoder.predict(X_train)
decoded_frames[0]

788/788 [==============================] - 4s 5ms/step


array([[[0.29616177, 0.29616177, 0.29616177],
        [0.2533188 , 0.2533188 , 0.2533188 ],
        [0.21353999, 0.21353999, 0.21353997],
        ...,
        [0.12446887, 0.12446887, 0.12446891],
        [0.12110503, 0.12110502, 0.12110506],
        [0.11860114, 0.1186011 , 0.11860114]],

       [[0.3028541 , 0.30285412, 0.30285412],
        [0.27956763, 0.27956763, 0.27956763],
        [0.23336981, 0.23336981, 0.23336981],
        ...,
        [0.12343405, 0.12343403, 0.12343403],
        [0.150978  , 0.15097794, 0.15097807],
        [0.11387522, 0.11387517, 0.11387524]],

       [[0.3104332 , 0.3104332 , 0.3104332 ],
        [0.27450374, 0.27450374, 0.27450374],
        [0.22957374, 0.22957374, 0.22957376],
        ...,
        [0.13026714, 0.13026705, 0.13026717],
        [0.14238207, 0.14238216, 0.14238214],
        [0.12258611, 0.12258614, 0.12258611]],

       ...,

       [[0.2981301 , 0.29813007, 0.2981301 ],
        [0.304994  , 0.304994  , 0.304994  ],
        [0.25623575, 0

In [11]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def show_frame(frame_index):
    plt.imshow(decoded_frames[frame_index].astype('uint8'))
    plt.axis('off')
    plt.show()

# Create a slider to browse through the frames
frame_slider = widgets.IntSlider(value=0, min=0, max=len(decoded_frames)-1, step=1, description='Frame:')

# Display the widget
widgets.interactive(show_frame, frame_index=frame_slider)


interactive(children=(IntSlider(value=0, description='Frame:', max=25199), Output()), _dom_classes=('widget-in…